In [2]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)

In [3]:
from financeta.clients.yfinance import YahooFinanceClient
yh = YahooFinanceClient()
df_init = yh.get_price("AAPL", from_date="2025-09-26", to_date="2025-10-04", interval="1m", postclean=True)

df_init.to_pandas()

,timestamp,datetime,open,high,low,close,volume
0,1758893400,2025-09-26 15:30:00,254.000000,254.919907,253.779999,254.569901,1502884
1,1758893460,2025-09-26 15:31:00,254.610001,254.860001,254.300095,254.850006,193864
2,1758893520,2025-09-26 15:32:00,254.880005,254.978699,254.500000,254.925003,260898
3,1758893580,2025-09-26 15:33:00,254.895203,255.059906,254.550003,254.839996,299410
4,1758893640,2025-09-26 15:34:00,254.843994,254.919998,254.520004,254.580002,161740
...,...,...,...,...,...,...,...
2333,1759521300,2025-10-03 21:55:00,257.700012,257.959991,257.640015,257.690002,618300
2334,1759521360,2025-10-03 21:56:00,257.695007,257.739990,257.600006,257.720001,155320
2335,1759521420,2025-10-03 21:57:00,257.711304,257.915009,257.660004,257.903503,231576
2336,1759521480,2025-10-03 21:58:00,257.899994,257.970001,257.850006,257.855011,346931


In [4]:
from financeta.utils.ta import SMA, RSI, MACD, TAClient
indicators = [
    SMA(10),
    SMA(20),
    RSI(14),
    MACD() 
]
ta_client = TAClient()
df = ta_client.calculate_indicators(df_init, indicators)

In [5]:
from financeta.utils.trace import Candlesticks, Volume, Line

traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]

In [ ]:
from financeta.clients.chart import ChartClient

chart_client = ChartClient()
chart_client.plot(df, "AAPL", traces=traces, indicators=indicators, theme='professional')

In [28]:
# test baysian optimization with skopt

import polars as pl
import numpy as np

def backtest_strategy(df: pl.DataFrame, short_window: int, long_window: int, rsi_period: int, stop_loss: float, take_profit: float) -> dict:
    """
    Version avec gestion d'erreurs robuste
    """
    try:
        df = df.clone().drop_nulls()
        
        if len(df) == 0:
            return {
                "total_return": 0.0,
                "sharpe_ratio": 0.0,
                "cagr": 0.0,
                "max_drawdown": 0.0
            }
        
        # Vérifier que les colonnes nécessaires existent
        required_cols = [f'SMA_{short_window}', f'SMA_{long_window}', f'RSI_{rsi_period}', 'close']
        missing_cols = [col for col in required_cols if col not in df.columns]
        
        if missing_cols:
            raise ValueError(f"Colonnes manquantes: {missing_cols}")
        
        # Générer les signaux
        df = df.with_columns([
            pl.when(
                (pl.col(f'SMA_{short_window}') > pl.col(f'SMA_{long_window}')) & 
                (pl.col(f'RSI_{rsi_period}') < 30)
            ).then(1)
            .when(
                (pl.col(f'SMA_{short_window}') < pl.col(f'SMA_{long_window}')) & 
                (pl.col(f'RSI_{rsi_period}') > 70)
            ).then(-1)
            .otherwise(0)
            .alias('signal')
        ])
        
        # Calculer les rendements
        df = df.with_columns([
            pl.col('close').pct_change().alias('price_returns'),
            pl.col('signal').shift(1).fill_null(0).alias('signal_shifted')
        ])
        
        df = df.with_columns([
            (pl.col('price_returns').fill_null(0) * pl.col('signal_shifted') - (stop_loss + take_profit))
            .alias('strategy_returns')
        ])
        
        # Calculer les métriques
        returns = df['strategy_returns'].drop_nulls()
        
        if len(returns) == 0:
            return {
                "total_return": 0.0,
                "sharpe_ratio": 0.0,
                "cagr": 0.0,
                "max_drawdown": 0.0
            }
        
        total_return = returns.sum()
        mean_ret = returns.mean()
        std_ret = returns.std()
        
        # Sharpe ratio
        sharpe_ratio = (mean_ret / std_ret) * np.sqrt(252 * 390) if std_ret and std_ret > 0 else 0.0
        
        # CAGR
        close_first = df['close'][0]
        close_last = df['close'][-1]
        n_periods = len(df)
        cagr = (close_last / close_first) ** (252 * 390 / n_periods) - 1 if n_periods > 0 and close_first > 0 else 0.0
        
        # Max Drawdown
        df = df.with_columns([
            ((1 + pl.col('strategy_returns')).cum_prod()).alias('cum_returns')
        ])
        df = df.with_columns([
            (pl.col('cum_returns') / pl.col('cum_returns').cum_max() - 1).alias('drawdown')
        ])
        
        max_drawdown = df['drawdown'].min() or 0.0
        
        return {
            "total_return": float(total_return),
            "sharpe_ratio": float(sharpe_ratio),
            "cagr": float(cagr),
            "max_drawdown": float(max_drawdown)
        }
        
    except Exception as e:
        print(f"Erreur dans backtest_strategy: {e}")
        return {
            "total_return": 0.0,
            "sharpe_ratio": 0.0,
            "cagr": 0.0,
            "max_drawdown": 0.0
        }

def objective(params):
    short_window, long_window, rsi_period, stop_loss, take_profit = params
    indicators = [
        SMA(short_window),
        SMA(long_window),
        RSI(),
    ]
    ta_client = TAClient()
    df = ta_client.calculate_indicators(df_init, indicators)

    # Backtest the strategy
    metrics = backtest_strategy(df, short_window, long_window, rsi_period, stop_loss, take_profit)
    
    # return minimize negative sharpe ratio
    return -metrics['sharpe_ratio']
    
    


In [29]:
# Configuration JSON avec structure logique
OPTIMIZATION_CONFIG = {
    "indicators": {
        "SMA_short": {
            "class": "SMA",
            "params": {
                "period": {
                    "type": "int",
                    "low": 5,
                    "high": 50
                }
            }
        },
        "SMA_long": {
            "class": "SMA",
            "params": {
                "period": {
                    "type": "int",
                    "low": 20,
                    "high": 200
                }
            }
        },
        "RSI": {
            "class": "RSI",
            "params": {
                "period": {
                    "type": "int",
                    "low": 7,
                    "high": 21
                }
            }
        },
        "BollingerBands": {
            "class": "BollingerBands",
            "params": {
                "period": {
                    "type": "int",
                    "low": 10,
                    "high": 30
                },
                "std_dev": {
                    "type": "float",
                    "low": 1.5,
                    "high": 3.0
                }
            }
        },
        "MACD": {
            "class": "MACD",
            "params": {
                "fast_period": {
                    "type": "int",
                    "low": 8,
                    "high": 16
                },
                "slow_period": {
                    "type": "int",
                    "low": 20,
                    "high": 30
                },
                "signal_period": {
                    "type": "int",
                    "low": 7,
                    "high": 12
                }
            }
        }
    },
    "strategy": {
        "stop_loss": {
            "type": "float",
            "low": 0.001,
            "high": 0.05,
            "log": True
        },
        "take_profit": {
            "type": "float",
            "low": 0.001,
            "high": 0.05,
            "log": True
        },
        "position_size": {
            "type": "float",
            "low": 0.1,
            "high": 1.0
        }
    },
    "constraints": [
        {
            "condition": "SMA_short_period < SMA_long_period"
        },
        {
            "condition": "MACD_fast_period < MACD_slow_period"
        }
    ],
    "optimization": {
        "direction": "maximize",
        "n_trials": 100,
        "n_jobs": 1,
        "timeout": 3600,
        "seed": 42
    }
}

In [30]:
from financeta.utils.ta import SMA, EMA, RSI, MACD, BollingerBands, ATR, Stochastic

# Mapping des classes d'indicateurs
INDICATOR_CLASSES = {
    'SMA': SMA,
    'EMA': EMA,
    'RSI': RSI,
    'MACD': MACD,
    'BollingerBands': BollingerBands,
    'ATR': ATR,
    'Stochastic': Stochastic,
}

In [31]:
from typing import Any, Dict, List
import optuna

def suggest_params_from_config(trial: optuna.Trial, config: Dict[str, Any]) -> Dict[str, Any]:
    """
    Génère les suggestions de paramètres pour les indicateurs et la stratégie
    
    Args:
        trial: Objet trial d'Optuna
        config: Configuration JSON complète
    
    Returns:
        Dictionnaire avec 'indicators' et 'strategy'
    """
    params = {
        'indicators': {},
        'strategy': {}
    }
    
    # Suggérer les paramètres des indicateurs
    for indicator_name, indicator_config in config.get("indicators", {}).items():
        params['indicators'][indicator_name] = {}
        
        for param_name, param_config in indicator_config.get("params", {}).items():
            param_type = param_config["type"]
            full_param_name = f"{indicator_name}_{param_name}"
            
            if param_type == "int":
                value = trial.suggest_int(
                    full_param_name,
                    param_config["low"],
                    param_config["high"],
                    step=param_config.get("step", 1)
                )
            elif param_type == "float":
                value = trial.suggest_float(
                    full_param_name,
                    param_config["low"],
                    param_config["high"],
                    log=param_config.get("log", False),
                    step=param_config.get("step", None)
                )
            elif param_type == "categorical":
                value = trial.suggest_categorical(
                    full_param_name,
                    param_config["choices"]
                )
            
            params['indicators'][indicator_name][param_name] = value
    
    # Suggérer les paramètres de stratégie
    for param_name, param_config in config.get("strategy", {}).items():
        param_type = param_config["type"]
        
        if param_type == "int":
            value = trial.suggest_int(
                param_name,
                param_config["low"],
                param_config["high"],
                step=param_config.get("step", 1)
            )
        elif param_type == "float":
            value = trial.suggest_float(
                param_name,
                param_config["low"],
                param_config["high"],
                log=param_config.get("log", False),
                step=param_config.get("step", None)
            )
        elif param_type == "categorical":
            value = trial.suggest_categorical(
                param_name,
                param_config["choices"]
            )
        
        params['strategy'][param_name] = value
    
    return params

def create_indicators_from_params(config: Dict[str, Any], params: Dict[str, Any]) -> List:
    """
    Crée automatiquement les indicateurs depuis les paramètres
    
    Args:
        config: Configuration JSON complète
        params: Paramètres suggérés (avec structure 'indicators' et 'strategy')
    
    Returns:
        Liste des objets indicateurs instanciés
    """
    indicators = []
    
    for indicator_name, indicator_params in params['indicators'].items():
        indicator_config = config['indicators'][indicator_name]
        indicator_class_name = indicator_config['class']
        
        if indicator_class_name not in INDICATOR_CLASSES:
            raise ValueError(f"Indicateur '{indicator_class_name}' non trouvé dans INDICATOR_CLASSES")
        
        indicator_class = INDICATOR_CLASSES[indicator_class_name]
        
        # Instancier l'indicateur avec les paramètres
        try:
            # Essayer d'abord avec kwargs
            indicator = indicator_class(**indicator_params)
        except TypeError:
            # Si ça échoue, essayer avec args positionnels
            indicator = indicator_class(*indicator_params.values())
        
        indicators.append(indicator)
    
    return indicators

def check_constraints(params: Dict[str, Any], constraints: list) -> bool:
    """
    Vérifie si les paramètres respectent les contraintes
    
    Args:
        params: Paramètres suggérés (structure plate pour l'évaluation)
        constraints: Liste des contraintes
    
    Returns:
        True si toutes les contraintes sont respectées
    """
    # Aplatir la structure des params pour l'évaluation
    flat_params = {}
    
    # Ajouter les paramètres d'indicateurs
    for indicator_name, indicator_params in params['indicators'].items():
        for param_name, param_value in indicator_params.items():
            flat_params[f"{indicator_name}_{param_name}"] = param_value
    
    # Ajouter les paramètres de stratégie
    flat_params.update(params['strategy'])
    
    # Vérifier les contraintes
    for constraint in constraints:
        condition = constraint["condition"]
        try:
            if not eval(condition, {}, flat_params):
                return False
        except Exception as e:
            print(f"Erreur lors de l'évaluation de la contrainte '{condition}': {e}")
            return False
    
    return True

In [32]:
from optuna.samplers import TPESampler
import json

def objective(trial):
    """
    Fonction objectif pour Optuna avec configuration JSON
    """
    config = OPTIMIZATION_CONFIG
    
    # Suggérer les paramètres depuis la config
    params = suggest_params_from_config(trial, config)
    
    # Vérifier les contraintes
    if not check_constraints(params, config.get("constraints", [])):
        return float('-inf')
    
    # Créer automatiquement les indicateurs depuis les paramètres
    indicators = create_indicators_from_params(config, params)
    
    print(indicators)
    
    # Debug: afficher les paramètres testés
    print(f"\nTrial {trial.number}:")
    print(f"  Indicators: {params['indicators']}")
    print(f"  Strategy: {params['strategy']}")
    
    # Calculer les indicateurs sur le DataFrame
    ta_client = TAClient()
    df = ta_client.calculate_indicators(df_init, indicators)
    
    # Backtest de la stratégie
    try:
        # Extraire les paramètres nécessaires pour le backtest
        # Adapter selon votre fonction backtest_strategy
        short_window = params['indicators']['SMA_short']['period']
        long_window = params['indicators']['SMA_long']['period']
        rsi_period = params['indicators']['RSI']['period']
        stop_loss = params['strategy']['stop_loss']
        take_profit = params['strategy']['take_profit']
        
        metrics = backtest_strategy(
            df, 
            short_window,
            long_window,
            rsi_period,
            stop_loss,
            take_profit
        )
        
        # Enregistrer les métriques
        trial.set_user_attr('total_return', metrics['total_return'])
        trial.set_user_attr('cagr', metrics['cagr'])
        trial.set_user_attr('max_drawdown', metrics['max_drawdown'])
        
        return metrics['sharpe_ratio']
    
    except Exception as e:
        print(f"  ❌ Erreur: {e}")
        return float('-inf')

def run_optimization(config: Dict[str, Any]):
    """
    Lance l'optimisation avec la configuration fournie
    """
    opt_config = config["optimization"]
    
    # Afficher la configuration
    print("="*70)
    print("CONFIGURATION DE L'OPTIMISATION")
    print("="*70)
    print(f"\nIndicateurs à optimiser:")
    for ind_name, ind_config in config['indicators'].items():
        print(f"  - {ind_name} ({ind_config['class']}): {list(ind_config['params'].keys())}")
    
    print(f"\nParamètres de stratégie:")
    for param_name in config['strategy'].keys():
        print(f"  - {param_name}")
    
    print(f"\nNombre de trials: {opt_config['n_trials']}")
    print(f"Direction: {opt_config['direction']}")
    print("="*70 + "\n")
    
    study = optuna.create_study(
        direction=opt_config["direction"],
        study_name='trading_strategy_optimization',
        sampler=TPESampler(seed=opt_config.get("seed", None)),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10)
    )
    
    study.optimize(
        objective, 
        n_trials=opt_config["n_trials"],
        n_jobs=opt_config.get("n_jobs", 1),
        show_progress_bar=True,
        timeout=opt_config.get("timeout", None)
    )
    
    print("\n" + "="*70)
    print("RÉSULTATS DE L'OPTIMISATION")
    print("="*70)
    print(f"\n🏆 Meilleur Sharpe Ratio: {study.best_value:.4f}")
    
    print(f"\n📊 Meilleurs paramètres:")
    print("\n  Indicateurs:")
    for key, value in study.best_params.items():
        if any(ind_name in key for ind_name in config['indicators'].keys()):
            print(f"    {key}: {value}")
    
    print("\n  Stratégie:")
    for key, value in study.best_params.items():
        if key in config['strategy'].keys():
            print(f"    {key}: {value}")
    
    print(f"\n💰 Métriques de performance:")
    print(f"    Total Return: {study.best_trial.user_attrs['total_return']:.4f}")
    print(f"    CAGR: {study.best_trial.user_attrs['cagr']:.4f}")
    print(f"    Max Drawdown: {study.best_trial.user_attrs['max_drawdown']:.4f}")
    
    return study


# Lancer l'optimisation
if __name__ == "__main__":
    # Option 1: Utiliser la config directement
    study = run_optimization(OPTIMIZATION_CONFIG)
    
    # Sauvegarder les résultats
    study.trials_dataframe().to_csv('optimization_results.csv', index=False)
    
    # Sauvegarder la meilleure config
    best_config = {
        'indicators': {},
        'strategy': {}
    }
    
    for key, value in study.best_params.items():
        if any(ind_name in key for ind_name in OPTIMIZATION_CONFIG['indicators'].keys()):
            # C'est un paramètre d'indicateur
            for ind_name in OPTIMIZATION_CONFIG['indicators'].keys():
                if key.startswith(ind_name):
                    if ind_name not in best_config['indicators']:
                        best_config['indicators'][ind_name] = {}
                    param_name = key.replace(f"{ind_name}_", "")
                    best_config['indicators'][ind_name][param_name] = value
        else:
            # C'est un paramètre de stratégie
            best_config['strategy'][key] = value
    
    with open('best_config.json', 'w') as f:
        json.dump(best_config, f, indent=4)
    
    print("\n✅ Meilleure configuration sauvegardée dans 'best_config.json'")

[I 2025-10-05 14:59:08,038] A new study created in memory with name: trading_strategy_optimization


CONFIGURATION DE L'OPTIMISATION

Indicateurs à optimiser:
  - SMA_short (SMA): ['period']
  - SMA_long (SMA): ['period']
  - RSI (RSI): ['period']
  - BollingerBands (BollingerBands): ['period', 'std_dev']
  - MACD (MACD): ['fast_period', 'slow_period', 'signal_period']

Paramètres de stratégie:
  - stop_loss
  - take_profit
  - position_size

Nombre de trials: 100
Direction: maximize



Best trial: 0. Best value: 0:  12%|█▏        | 12/100 [00:00<00:01, 78.41it/s, 0.19/3600 seconds]

[<financeta.utils.ta.SMA object at 0x7f07f3734ad0>, <financeta.utils.ta.SMA object at 0x7f07f3734d50>, <financeta.utils.ta.RSI object at 0x7f07f3734490>, <financeta.utils.ta.BollingerBands object at 0x7f07f2178750>, <financeta.utils.ta.MACD object at 0x7f07f2179a90>]

Trial 0:
  Indicators: {'SMA_short': {'period': 22}, 'SMA_long': {'period': 192}, 'RSI': {'period': 17}, 'BollingerBands': {'period': 22, 'std_dev': 1.7340279606636548}, 'MACD': {'fast_period': 9, 'slow_period': 20, 'signal_period': 12}}
  Strategy: {'stop_loss': 0.010502105436744277, 'take_profit': 0.01595857358814127, 'position_size': 0.1185260448662222}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_22', 'SMA_192', 'RSI_17']
[I 2025-10-05 14:59:08,050] Trial 0 finished with value: 0.0 and parameters: {'SMA_short_period': 22, 'SMA_long_period': 192, 'RSI_period': 17, 'BollingerBands_period': 22, 'BollingerBands_std_dev': 1.7340279606636548, 'MACD_fast_period': 9, 'MACD_slow_period': 20, 'MACD_signal_period': 

Best trial: 0. Best value: 0:  19%|█▉        | 19/100 [00:00<00:01, 51.22it/s, 0.38/3600 seconds]

...

[<financeta.utils.ta.SMA object at 0x7f07914055d0>, <financeta.utils.ta.SMA object at 0x7f07f2ccd4d0>, <financeta.utils.ta.RSI object at 0x7f07f0935610>, <financeta.utils.ta.BollingerBands object at 0x7f07c911d450>, <financeta.utils.ta.MACD object at 0x7f07c911d810>]

Trial 85:
  Indicators: {'SMA_short': {'period': 48}, 'SMA_long': {'period': 162}, 'RSI': {'period': 18}, 'BollingerBands': {'period': 17, 'std_dev': 2.25413983104258}, 'MACD': {'fast_period': 13, 'slow_period': 20, 'signal_period': 9}}
  Strategy: {'stop_loss': 0.001038526520288999, 'take_profit': 0.0017109875435938925, 'position_size': 0.12083285350492802}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_48', 'SMA_162', 'RSI_18']
[I 2025-10-05 14:59:10,381] Trial 85 finished with value: 0.0 and parameters: {'SMA_short_period': 48, 'SMA_long_period': 162, 'RSI_period': 18, 'BollingerBands_period': 17, 'BollingerBands_std_dev': 2.25413983104258, 'MACD_fast_period': 13, 'MACD_slow_period': 20, 'MACD_signal_period'

Best trial: 0. Best value: 0:  98%|█████████▊| 98/100 [00:02<00:00, 31.54it/s, 2.74/3600 seconds]

[<financeta.utils.ta.SMA object at 0x7f07f21799d0>, <financeta.utils.ta.SMA object at 0x7f07bb8e7090>, <financeta.utils.ta.RSI object at 0x7f07c918c290>, <financeta.utils.ta.BollingerBands object at 0x7f07c91f1a10>, <financeta.utils.ta.MACD object at 0x7f07c9191850>]

Trial 91:
  Indicators: {'SMA_short': {'period': 49}, 'SMA_long': {'period': 156}, 'RSI': {'period': 20}, 'BollingerBands': {'period': 21, 'std_dev': 1.663311447501595}, 'MACD': {'fast_period': 8, 'slow_period': 23, 'signal_period': 12}}
  Strategy: {'stop_loss': 0.0020742984639249896, 'take_profit': 0.04740033913774456, 'position_size': 0.2002227304825781}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_49', 'SMA_156', 'RSI_20']
[I 2025-10-05 14:59:10,584] Trial 91 finished with value: 0.0 and parameters: {'SMA_short_period': 49, 'SMA_long_period': 156, 'RSI_period': 20, 'BollingerBands_period': 21, 'BollingerBands_std_dev': 1.663311447501595, 'MACD_fast_period': 8, 'MACD_slow_period': 23, 'MACD_signal_period':

Best trial: 0. Best value: 0: 100%|██████████| 100/100 [00:02<00:00, 35.64it/s, 2.80/3600 seconds]


[<financeta.utils.ta.SMA object at 0x7f07c918f190>, <financeta.utils.ta.SMA object at 0x7f07914053d0>, <financeta.utils.ta.RSI object at 0x7f07c91df0d0>, <financeta.utils.ta.BollingerBands object at 0x7f07c91df150>, <financeta.utils.ta.MACD object at 0x7f07c91df350>]

Trial 98:
  Indicators: {'SMA_short': {'period': 20}, 'SMA_long': {'period': 174}, 'RSI': {'period': 20}, 'BollingerBands': {'period': 19, 'std_dev': 2.0921666176259377}, 'MACD': {'fast_period': 10, 'slow_period': 25, 'signal_period': 10}}
  Strategy: {'stop_loss': 0.0012513622766875047, 'take_profit': 0.0048511141346914135, 'position_size': 0.11725601325805615}
Erreur dans backtest_strategy: Colonnes manquantes: ['SMA_20', 'SMA_174', 'RSI_20']
[I 2025-10-05 14:59:10,809] Trial 98 finished with value: 0.0 and parameters: {'SMA_short_period': 20, 'SMA_long_period': 174, 'RSI_period': 20, 'BollingerBands_period': 19, 'BollingerBands_std_dev': 2.0921666176259377, 'MACD_fast_period': 10, 'MACD_slow_period': 25, 'MACD_signal_p

In [33]:
best_config

{'indicators': {'SMA_short': {'period': 22},
  'SMA_long': {'period': 192},
  'RSI': {'period': 17},
  'BollingerBands': {'period': 22, 'std_dev': 1.7340279606636548},
  'MACD': {'fast_period': 9, 'slow_period': 20, 'signal_period': 12}},
 'strategy': {'stop_loss': 0.010502105436744277,
  'take_profit': 0.01595857358814127,
  'position_size': 0.1185260448662222}}

In [ ]:
# crée un plot avec les meilleurs paramètres
best_indicators = create_indicators_from_params(OPTIMIZATION_CONFIG, {
    'indicators': best_config['indicators'],
    'strategy': best_config['strategy']
})
traces = [
    Candlesticks(),
    Line('hl_avg', name='HL Average', color='purple'),
    Volume()
]
chart_client.plot(df, "AAPL - Best Config", traces=traces, indicators=best_indicators, theme='professional')